In [1]:
#!pip install pandas --quiet
#!pip install xgboost --quiet
#!pip install -U scikit-learn --quiet
#!pip install cupy-cuda12x --quiet
#!pip install optuna --quiet
#!pip install kaggle --quiet

In [1]:
# Print informatbion abput the GPU and CPU
!nvidia-smi

Tue Apr 16 10:42:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:03:00.0 Off |                  N/A |
|  0%   39C    P8              19W / 200W |      1MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from utils import *
# optimization
import optuna

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)



In [3]:
def print_trial_info(study, trial):
    print(f"Trial {trial.number}: Value = {trial.value}")

pruner = optuna.pruners.SuccessiveHalvingPruner()
study = optuna.create_study(direction='minimize', study_name = "Xgboost forecast finetuning")
study.optimize(objective, n_trials=100)

[I 2024-04-16 10:42:29,969] A new study created in memory with name: Xgboost forecast finetuning
[I 2024-04-16 10:46:32,201] Trial 0 finished with value: 0.017150961066055303 and parameters: {'lag': 250, 'max_depth': 17, 'n_estimators': 700, 'gamma': 0.1, 'subsample': 0.7, 'colsample_bytree': 0.8, 'reg_alpha': 0.0011768440213115923, 'reg_lambda': 0.00011122275098108434, 'learning_rate': 0.008677048247032382}. Best is trial 0 with value: 0.017150961066055303.
[I 2024-04-16 10:49:24,421] Trial 1 finished with value: 0.015163473068542483 and parameters: {'lag': 190, 'max_depth': 11, 'n_estimators': 600, 'gamma': 0.6, 'subsample': 0.6, 'colsample_bytree': 0.7, 'reg_alpha': 0.000630001991670398, 'reg_lambda': 0.06633905521845301, 'learning_rate': 0.003323752773862041}. Best is trial 1 with value: 0.015163473068542483.
[I 2024-04-16 10:51:17,439] Trial 2 finished with value: 0.020365593480911257 and parameters: {'lag': 130, 'max_depth': 4, 'n_estimators': 500, 'gamma': 0.4, 'subsample': 0.9,

KeyboardInterrupt: 

In [5]:
best_params = study.best_params
best_params

{'lag': 280,
 'max_depth': 16,
 'n_estimators': 1200,
 'gamma': 0.1,
 'subsample': 0.5,
 'colsample_bytree': 0.6,
 'reg_alpha': 2.638488376439401e-05,
 'reg_lambda': 0.00036267566450740247,
 'learning_rate': 0.0005324873278995036}

In [6]:
xgb = XGBRegressor(tree_method="hist", random_state=random_seed, device='cuda', eval_metric = 'mae',
                       max_depth=best_params['max_depth'], n_estimators=best_params['n_estimators'], gamma=best_params['gamma'], 
                       subsample=best_params['subsample'], colsample_bytree=best_params['colsample_bytree'],
                       reg_alpha=best_params['reg_alpha'], reg_lambda=best_params['reg_lambda'], learning_rate=best_params['learning_rate'])

_, X_test, _, y_test = create_data(df, lags=best_params['lag'], steps = 99, y_columns=['6'], test_size=0, cuda=True)
X_train = X_test
y_train = y_test

xgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0005324873278995036,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=16, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [7]:
x_forecaste = create_lags(df, lags=best_params['lag']-1).iloc[-1]
x_forecaste = x_forecaste.to_numpy()
x_forecaste = np.expand_dims(x_forecaste, axis=0)
y_forecast = xgb.predict(cp.array(x_forecaste).astype(cp.float32)).flatten()
y_forecast = np.array(y_forecast)

x_forecaste.shape,y_forecast.shape

((1, 2240), (100,))

## save prediction

In [10]:
submission = pd.DataFrame(y_forecast, columns=['Prediction'])
submission['Id'] = range(100)
submission = submission[['Id', 'Prediction']]
submission.to_csv('submission.csv', index=False)